## Spotify API 搜尋熱門、非熱門歌曲特徵

In [1]:
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
# cid, secret 為 spotify for developers 之帳號密碼
cid = 'cid'
secret = 'secret'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### 熱門歌曲

In [2]:
# 讀取排行榜歌曲清單
hotsongs = pd.read_csv('熱門歌曲清單.csv')
hotsongs.head()

,名次,歌名,歌手,語言,年分,專輯
0,1,愛情你比我想的閣較偉大,茄子蛋,華語,2021,《當男人戀愛時》電影主題曲
1,2,在這座城市遺失了你,告五人,華語,2021,《運氣來得若有似無》
2,3,我很好騙,動力火車,華語,2021,《都是因為愛》
3,4,四季予你,程響,華語,2021,《四季予你》
4,5,阿拉斯加海灣,蔡恩雨,華語,2021,《阿拉斯加海灣》


In [3]:
hotsongs.columns = ["rank", "song", "singer", "language", "year", "album"]
hotsongs['album'] = hotsongs['album'].str[1:-1]
hotsongs.head()

,rank,song,singer,language,year,album
0,1,愛情你比我想的閣較偉大,茄子蛋,華語,2021,當男人戀愛時》電影主題
1,2,在這座城市遺失了你,告五人,華語,2021,運氣來得若有似無
2,3,我很好騙,動力火車,華語,2021,都是因為愛
3,4,四季予你,程響,華語,2021,四季予你
4,5,阿拉斯加海灣,蔡恩雨,華語,2021,阿拉斯加海灣


In [5]:
# 檢查缺失值
hotsongs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10892 entries, 0 to 10891
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   rank      10892 non-null  int64 
 1   song      10892 non-null  object
 2   singer    10892 non-null  object
 3   language  10892 non-null  object
 4   year      10892 non-null  int64 
 5   album     6249 non-null   object
dtypes: int64(2), object(4)
memory usage: 510.7+ KB


In [ ]:
artist_name_hot = []
track_name_hot = []
track_id_hot = []
album_name_hot = []
popularity_hot = []

# 使用「歌手」及「歌曲」查詢熱門歌曲資料
for i in range(len(hotsongs)):
    q_artist = 'artist:' + hotsongs["singer"][i]
    q_track = 'track:' + hotsongs["song"][i]
    track_results = sp.search(q=q_artist+' '+q_track, type='track', market="TW", limit=1, offset=0)
    
    if(len(track_results['tracks']['items']) == 0):
        artist_name_hot.append('no result')
        track_name_hot.append('no result')
        track_id_hot.append('no result')
        album_name_hot.append('no result')
        popularity_hot.append('no result')
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name_hot.append(t['artists'][0]['name'])
        track_name_hot.append(t['name'])
        track_id_hot.append(t['id'])
        album_name_hot.append(t['album']['name'])
        popularity_hot.append(t['popularity'])

In [7]:
track_hot = pd.DataFrame({'artist_name' : artist_name_hot, 'track_name' : track_name_hot, 'track_id' : track_id_hot, 'album_name' : album_name_hot, 'popularity' : popularity_hot})
track_hot

,artist_name,track_name,track_id,album_name,popularity
0,no result,no result,no result,no result,no result
1,告五人,在這座城市遺失了你 (戲劇《他們創業的那些鳥事》插曲),1Ytgo9ipdlTsf6wlg6sXf3,運氣來得若有似無,60
2,no result,no result,no result,no result,no result
3,程響,四季予你,4BGkSCrjiLao1bpibNptTD,四季予你,59
4,no result,no result,no result,no result,no result
...,...,...,...,...,...
10887,no result,no result,no result,no result,no result
10888,no result,no result,no result,no result,no result
10889,no result,no result,no result,no result,no result
10890,SHINee,Up & Down,0Ds1F3FapeVqxZYPQ3b6Ih,Lucifer - The 2nd Album,31


In [8]:
# 合併排行榜及 spotify 查詢結果
df_hot = hotsongs.join(track_hot)
# 去除無搜尋結果的資料
df_hot = df_hot[df_hot["artist_name"] != "no result"]
df_hot = df_hot[["year", "language", "artist_name", "track_name", "track_id", "album_name", "popularity"]]
df_hot = df_hot.reset_index(drop=True)
df_hot

,year,language,artist_name,track_name,track_id,album_name,popularity
0,2021,華語,告五人,在這座城市遺失了你 (戲劇《他們創業的那些鳥事》插曲),1Ytgo9ipdlTsf6wlg6sXf3,運氣來得若有似無,60
1,2021,華語,程響,四季予你,4BGkSCrjiLao1bpibNptTD,四季予你,59
2,2021,華語,Eric Chou,如果能幸福 - HBO Asia原創影集《戒指流浪記》片尾曲,0TVrlU8El0K6U1pC2LOt09,如果能幸福 (HBO Asia原創影集《戒指流浪記》片尾曲),56
3,2021,華語,WeiBird,"如果可以 - 電影""月老""主題曲",72OVnXDzugvrCU25lMi9au,"如果可以 (電影""月老""主題曲)",71
4,2021,華語,告五人,唯一 (三立/台視戲劇《戀愛是科學》插曲),19fp9nI0tq0lcBl7XoCHAb,運氣來得若有似無,62
...,...,...,...,...,...,...,...
4174,2010,西洋,P!nk,Raise Your Glass,1gv4xPanImH17bKZ9rOveR,Raise Your Glass,81
4175,2010,日語,2PM,Still,3cqA8LZBFfioCQ32vdqPhN,Still 2:00pm,8
4176,2010,日語,2PM,I know,6aFmnIb0yJ617c39j5qFJQ,2PM OF 2PM <リパッケージ>,15
4177,2010,日語,2PM,Dance2Night,6hS2alP36VK5UvkpdgWJdH,Still 2:00pm,6


In [9]:
def song_attribute(row):
    if (row["artist_name"] != "no result"):
        song_results = sp.audio_features(row["track_id"])
        df_song_result = pd.DataFrame(song_results)
        
        if "danceability" in df_song_result:
            row["danceability"] = df_song_result["danceability"][0]
        if "energy" in df_song_result:
            row["energy"] = df_song_result["energy"][0]
        if "key" in df_song_result:
            row["key"] = df_song_result["key"][0]
        if "loudness" in df_song_result:
            row["loudness"] = df_song_result["loudness"][0]
        if "mode" in df_song_result:
            row["mode"] = df_song_result["mode"][0]
        if "speechiness" in df_song_result:
            row["speechiness"] = df_song_result["speechiness"][0]
        if "acousticness" in df_song_result:
            row["acousticness"] = df_song_result["acousticness"][0]
        if "instrumentalness" in df_song_result:
            row["instrumentalness"] = df_song_result["instrumentalness"][0]
        if "liveness" in df_song_result:
            row["liveness"] = df_song_result["liveness"][0]
        if "valence" in df_song_result:
            row["valence"] = df_song_result["valence"][0]
        if "tempo" in df_song_result:
            row["tempo"] = df_song_result["tempo"][0]
        if "duration_ms" in df_song_result:
            row["duration(s)"] = df_song_result["duration_ms"][0] / 1000
        if "time_signature" in df_song_result:
            row["time_signature"] = df_song_result["time_signature"][0]

    return row

In [10]:
# 取得歌曲特徵值
df_hot = df_hot.apply(song_attribute, axis=1)
df_hot.head()

,year,language,artist_name,track_name,track_id,album_name,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration(s),time_signature
0,2021,華語,告五人,在這座城市遺失了你 (戲劇《他們創業的那些鳥事》插曲),1Ytgo9ipdlTsf6wlg6sXf3,運氣來得若有似無,60,0.460,0.297,9,-8.785,1,0.0309,0.692,0.000318,0.107,0.125,180.095,357.094,4
1,2021,華語,程響,四季予你,4BGkSCrjiLao1bpibNptTD,四季予你,59,0.534,0.595,3,-6.745,1,0.0330,0.750,0.000000,0.116,0.187,141.957,248.527,4
2,2021,華語,Eric Chou,如果能幸福 - HBO Asia原創影集《戒指流浪記》片尾曲,0TVrlU8El0K6U1pC2LOt09,如果能幸福 (HBO Asia原創影集《戒指流浪記》片尾曲),56,0.480,0.358,0,-9.565,1,0.0282,0.622,0.000021,0.107,0.142,149.945,259.588,4
3,2021,華語,WeiBird,"如果可以 - 電影""月老""主題曲",72OVnXDzugvrCU25lMi9au,"如果可以 (電影""月老""主題曲)",71,0.540,0.540,3,-6.368,1,0.0290,0.241,0.000000,0.351,0.372,74.969,274.400,4
4,2021,華語,告五人,唯一 (三立/台視戲劇《戀愛是科學》插曲),19fp9nI0tq0lcBl7XoCHAb,運氣來得若有似無,62,0.485,0.103,1,-13.383,1,0.0367,0.959,0.000053,0.107,0.210,140.595,270.761,4


In [11]:
# 設熱門歌曲為陽性
df_hot = df_hot.assign(label = 1)
df_hot.head()

,year,language,artist_name,track_name,track_id,album_name,popularity,danceability,energy,key,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration(s),time_signature,label
0,2021,華語,告五人,在這座城市遺失了你 (戲劇《他們創業的那些鳥事》插曲),1Ytgo9ipdlTsf6wlg6sXf3,運氣來得若有似無,60,0.460,0.297,9,...,1,0.0309,0.692,0.000318,0.107,0.125,180.095,357.094,4,1
1,2021,華語,程響,四季予你,4BGkSCrjiLao1bpibNptTD,四季予你,59,0.534,0.595,3,...,1,0.0330,0.750,0.000000,0.116,0.187,141.957,248.527,4,1
2,2021,華語,Eric Chou,如果能幸福 - HBO Asia原創影集《戒指流浪記》片尾曲,0TVrlU8El0K6U1pC2LOt09,如果能幸福 (HBO Asia原創影集《戒指流浪記》片尾曲),56,0.480,0.358,0,...,1,0.0282,0.622,0.000021,0.107,0.142,149.945,259.588,4,1
3,2021,華語,WeiBird,"如果可以 - 電影""月老""主題曲",72OVnXDzugvrCU25lMi9au,"如果可以 (電影""月老""主題曲)",71,0.540,0.540,3,...,1,0.0290,0.241,0.000000,0.351,0.372,74.969,274.400,4,1
4,2021,華語,告五人,唯一 (三立/台視戲劇《戀愛是科學》插曲),19fp9nI0tq0lcBl7XoCHAb,運氣來得若有似無,62,0.485,0.103,1,...,1,0.0367,0.959,0.000053,0.107,0.210,140.595,270.761,4,1


### 非熱門歌曲

In [ ]:
artist_name_cold = []
track_name_cold = []
track_id_cold = []
album_name_cold = []
popularity_cold = []

# 使用「年分」及「歌手」查詢非熱門歌曲資料
for i in range(len(df_hot)): 

    artist_name_temp = []
    track_name_temp = []
    track_id_temp = []
    album_name_temp = []
    popularity_temp = []
    
    q_year = 'year:' + str(df_hot["year"][i])
    q_artist = 'artist:' + df_hot["artist_name"][i]
    track_results = sp.search(q=q_year+' '+q_artist, type='track', market="TW", limit=50, offset=0) #=q_year+' '+

    for k, t in enumerate(track_results['tracks']['items']):
        artist_name_temp.append(t['artists'][0]['name'])
        track_name_temp.append(t['name'])
        track_id_temp.append(t['id'])
        album_name_temp.append(t['album']['name'])
        popularity_temp.append(t['popularity'])
        
    
    min_index = 0
    min_popularity = 101
    
    for k in range(len(track_name_temp)):
        if (track_name_temp[k] in df_hot["track_name"].tolist() or 
            track_name_temp[k] in track_name_cold):
            continue
        else:
            if (popularity_temp[k] < min_popularity):
                min_index = k
                min_popularity = popularity_temp[k]
    
    
    if (min_popularity < 101):
        artist_name_cold.append(artist_name_temp[min_index])
        track_name_cold.append(track_name_temp[min_index])
        track_id_cold.append(track_id_temp[min_index])
        album_name_cold.append(album_name_temp[min_index])
        popularity_cold.append(popularity_temp[min_index])
 
    else:
        artist_name_cold.append('no result')
        track_name_cold.append('no result')
        track_id_cold.append('no result')
        album_name_cold.append('no result')
        popularity_cold.append('no result')

In [8]:
track_cold = pd.DataFrame({'year' : df_hot["year"].to_list(), 'language' : df_hot["language"].to_list(), 'artist_name' : artist_name_cold, 'track_name' : track_name_cold, 'track_id' : track_id_cold, 'album_name' : album_name_cold, 'popularity' : popularity_cold})
track_cold

,year,language,artist_name,track_name,track_id,album_name,popularity
0,2021,華語,告五人,只管向前 (2021黑松沙士年度主題曲),5ko89DUFQquXcReZAn1Dfy,只管向前,37
1,2021,華語,程響,君不知,2RT850OJa0DIZ0I2NaUup8,长安伏妖 电影音乐原声带,3
2,2021,華語,Eric Gemsa,Choubidoo Scat,63h1nLwsbakd7PfKFH0HGM,Riverside Walk,0
3,2021,華語,WeiBird,Is It Not Enough,3ATNtecQvkSs7dWHqCmosj,I'M MORE SOBER WHEN I'M DRUNK,27
4,2021,華語,告五人,Live A Little (Safari Club 調酒主題曲),4BA7ORrBhaBfiepX6fUKj5,Live A Little,38
...,...,...,...,...,...,...,...
4152,2010,西洋,Prophetess Helen Nkume,Adam Olee Ebe Ino ?,3HF2QQOZC4MHRuMQA8MyrM,Adam Olee Ebe Ino ?,6
4153,2010,日語,Sunday 2PM,돌연변이,6WmL0e1FXxUjpwZKMThrVP,돌연변이,0
4154,2010,日語,Sunday 2PM,어떻게,6FR2t0ytRuak9p5bcZJTGQ,돌연변이,0
4155,2010,日語,Sunday 2PM,기도,3fIER6IBdl7GnQrZNFuoiE,돌연변이,0


In [13]:
# 去除無搜尋結果的資料
df_cold = track_cold[track_cold["artist_name"] != "no result"]
df_cold

,year,language,artist_name,track_name,track_id,album_name,popularity
0,2021,華語,告五人,只管向前 (2021黑松沙士年度主題曲),5ko89DUFQquXcReZAn1Dfy,只管向前,37
1,2021,華語,程響,君不知,2RT850OJa0DIZ0I2NaUup8,长安伏妖 电影音乐原声带,3
2,2021,華語,Eric Gemsa,Choubidoo Scat,63h1nLwsbakd7PfKFH0HGM,Riverside Walk,0
3,2021,華語,WeiBird,Is It Not Enough,3ATNtecQvkSs7dWHqCmosj,I'M MORE SOBER WHEN I'M DRUNK,27
4,2021,華語,告五人,Live A Little (Safari Club 調酒主題曲),4BA7ORrBhaBfiepX6fUKj5,Live A Little,38
...,...,...,...,...,...,...,...
4152,2010,西洋,Prophetess Helen Nkume,Adam Olee Ebe Ino ?,3HF2QQOZC4MHRuMQA8MyrM,Adam Olee Ebe Ino ?,6
4153,2010,日語,Sunday 2PM,돌연변이,6WmL0e1FXxUjpwZKMThrVP,돌연변이,0
4154,2010,日語,Sunday 2PM,어떻게,6FR2t0ytRuak9p5bcZJTGQ,돌연변이,0
4155,2010,日語,Sunday 2PM,기도,3fIER6IBdl7GnQrZNFuoiE,돌연변이,0


In [18]:
# 取得歌曲特徵值
df_cold = df_cold.apply(song_attribute, axis=1)
df_cold.head()

,acousticness,album_name,artist_name,danceability,duration(s),energy,instrumentalness,key,language,liveness,loudness,mode,popularity,speechiness,tempo,time_signature,track_id,track_name,valence,year
0,0.04160,只管向前,告五人,0.305,254.922,0.670,0.000039,0.0,華語,0.2020,-4.805,1.0,37,0.0355,179.852,4.0,5ko89DUFQquXcReZAn1Dfy,只管向前 (2021黑松沙士年度主題曲),0.3240,2021
1,0.45500,长安伏妖 电影音乐原声带,程響,0.220,233.643,0.512,0.000003,8.0,華語,0.0821,-8.140,0.0,3,0.0369,173.054,4.0,2RT850OJa0DIZ0I2NaUup8,君不知,0.2840,2021
2,0.44700,Riverside Walk,Eric Gemsa,0.718,108.159,0.618,0.028800,11.0,華語,0.0532,-5.767,0.0,0,0.0973,92.173,4.0,63h1nLwsbakd7PfKFH0HGM,Choubidoo Scat,0.3880,2021
3,0.00408,I'M MORE SOBER WHEN I'M DRUNK,WeiBird,0.432,170.309,0.470,0.000050,8.0,華語,0.1440,-9.107,0.0,27,0.0515,117.218,3.0,3ATNtecQvkSs7dWHqCmosj,Is It Not Enough,0.0686,2021
4,0.00870,Live A Little,告五人,0.657,244.000,0.821,0.000005,9.0,華語,0.0744,-4.629,1.0,38,0.0469,120.021,4.0,4BA7ORrBhaBfiepX6fUKj5,Live A Little (Safari Club 調酒主題曲),0.5600,2021


In [19]:
# 設非熱門歌曲為陰性
df_cold = df_cold.assign(label = 0)
df_cold.head()

,acousticness,album_name,artist_name,danceability,duration(s),energy,instrumentalness,key,language,liveness,...,mode,popularity,speechiness,tempo,time_signature,track_id,track_name,valence,year,label
0,0.04160,只管向前,告五人,0.305,254.922,0.670,0.000039,0.0,華語,0.2020,...,1.0,37,0.0355,179.852,4.0,5ko89DUFQquXcReZAn1Dfy,只管向前 (2021黑松沙士年度主題曲),0.3240,2021,0
1,0.45500,长安伏妖 电影音乐原声带,程響,0.220,233.643,0.512,0.000003,8.0,華語,0.0821,...,0.0,3,0.0369,173.054,4.0,2RT850OJa0DIZ0I2NaUup8,君不知,0.2840,2021,0
2,0.44700,Riverside Walk,Eric Gemsa,0.718,108.159,0.618,0.028800,11.0,華語,0.0532,...,0.0,0,0.0973,92.173,4.0,63h1nLwsbakd7PfKFH0HGM,Choubidoo Scat,0.3880,2021,0
3,0.00408,I'M MORE SOBER WHEN I'M DRUNK,WeiBird,0.432,170.309,0.470,0.000050,8.0,華語,0.1440,...,0.0,27,0.0515,117.218,3.0,3ATNtecQvkSs7dWHqCmosj,Is It Not Enough,0.0686,2021,0
4,0.00870,Live A Little,告五人,0.657,244.000,0.821,0.000005,9.0,華語,0.0744,...,1.0,38,0.0469,120.021,4.0,4BA7ORrBhaBfiepX6fUKj5,Live A Little (Safari Club 調酒主題曲),0.5600,2021,0


In [20]:
# 合併熱門與非熱門歌曲資料
df_all = df_hot.append(df_cold, ignore_index=True)
df_all

C:\Users\Nicole\AppData\Local\Temp\ipykernel_14540\2485070849.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_hot.append(df_cold, ignore_index=True)


,year,language,artist_name,track_name,track_id,album_name,popularity,danceability,energy,key,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration(s),time_signature,label
0,2021,華語,告五人,在這座城市遺失了你 (戲劇《他們創業的那些鳥事》插曲),1Ytgo9ipdlTsf6wlg6sXf3,運氣來得若有似無,60,0.460,0.297,9.0,...,1.0,0.0309,0.6920,0.000318,0.1070,0.125,180.095,357.094,4.0,1
1,2021,華語,程響,四季予你,4BGkSCrjiLao1bpibNptTD,四季予你,59,0.534,0.595,3.0,...,1.0,0.0330,0.7500,0.000000,0.1160,0.187,141.957,248.527,4.0,1
2,2021,華語,Eric Chou,如果能幸福 - HBO Asia原創影集《戒指流浪記》片尾曲,0TVrlU8El0K6U1pC2LOt09,如果能幸福 (HBO Asia原創影集《戒指流浪記》片尾曲),55,0.480,0.358,0.0,...,1.0,0.0282,0.6220,0.000021,0.1070,0.142,149.945,259.588,4.0,1
3,2021,華語,WeiBird,"如果可以 - 電影""月老""主題曲",72OVnXDzugvrCU25lMi9au,"如果可以 (電影""月老""主題曲)",71,0.540,0.540,3.0,...,1.0,0.0290,0.2410,0.000000,0.3510,0.372,74.969,274.400,4.0,1
4,2021,華語,告五人,唯一 (三立/台視戲劇《戀愛是科學》插曲),19fp9nI0tq0lcBl7XoCHAb,運氣來得若有似無,62,0.485,0.103,1.0,...,1.0,0.0367,0.9590,0.000053,0.1070,0.210,140.595,270.761,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7857,2010,西洋,Prophetess Helen Nkume,Adam Olee Ebe Ino ?,3HF2QQOZC4MHRuMQA8MyrM,Adam Olee Ebe Ino ?,6,0.774,0.860,10.0,...,0.0,0.1730,0.1630,0.001000,0.0605,0.760,100.102,805.280,4.0,0
7858,2010,日語,Sunday 2PM,돌연변이,6WmL0e1FXxUjpwZKMThrVP,돌연변이,0,0.633,0.914,1.0,...,1.0,0.2560,0.0667,0.000000,0.2990,0.769,107.300,185.222,4.0,0
7859,2010,日語,Sunday 2PM,어떻게,6FR2t0ytRuak9p5bcZJTGQ,돌연변이,0,0.657,0.822,11.0,...,0.0,0.2320,0.0856,0.000000,0.0791,0.563,95.941,226.548,4.0,0
7860,2010,日語,Sunday 2PM,기도,3fIER6IBdl7GnQrZNFuoiE,돌연변이,0,0.385,0.856,11.0,...,0.0,0.2340,0.2410,0.000000,0.3650,0.519,86.628,291.070,5.0,0


In [21]:
# 檢查缺失值
print(df_all.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7862 entries, 0 to 7861
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   year              7862 non-null   int64  
 1   language          7862 non-null   object 
 2   artist_name       7862 non-null   object 
 3   track_name        7862 non-null   object 
 4   track_id          7862 non-null   object 
 5   album_name        7862 non-null   object 
 6   popularity        7862 non-null   int64  
 7   danceability      7861 non-null   float64
 8   energy            7861 non-null   float64
 9   key               7861 non-null   float64
 10  loudness          7861 non-null   float64
 11  mode              7861 non-null   float64
 12  speechiness       7861 non-null   float64
 13  acousticness      7861 non-null   float64
 14  instrumentalness  7861 non-null   float64
 15  liveness          7861 non-null   float64
 16  valence           7861 non-null   float64


In [22]:
# 去除包含缺失值的資料
df_all = df_all.dropna()
df_all

,year,language,artist_name,track_name,track_id,album_name,popularity,danceability,energy,key,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration(s),time_signature,label
0,2021,華語,告五人,在這座城市遺失了你 (戲劇《他們創業的那些鳥事》插曲),1Ytgo9ipdlTsf6wlg6sXf3,運氣來得若有似無,60,0.460,0.297,9.0,...,1.0,0.0309,0.6920,0.000318,0.1070,0.125,180.095,357.094,4.0,1
1,2021,華語,程響,四季予你,4BGkSCrjiLao1bpibNptTD,四季予你,59,0.534,0.595,3.0,...,1.0,0.0330,0.7500,0.000000,0.1160,0.187,141.957,248.527,4.0,1
2,2021,華語,Eric Chou,如果能幸福 - HBO Asia原創影集《戒指流浪記》片尾曲,0TVrlU8El0K6U1pC2LOt09,如果能幸福 (HBO Asia原創影集《戒指流浪記》片尾曲),55,0.480,0.358,0.0,...,1.0,0.0282,0.6220,0.000021,0.1070,0.142,149.945,259.588,4.0,1
3,2021,華語,WeiBird,"如果可以 - 電影""月老""主題曲",72OVnXDzugvrCU25lMi9au,"如果可以 (電影""月老""主題曲)",71,0.540,0.540,3.0,...,1.0,0.0290,0.2410,0.000000,0.3510,0.372,74.969,274.400,4.0,1
4,2021,華語,告五人,唯一 (三立/台視戲劇《戀愛是科學》插曲),19fp9nI0tq0lcBl7XoCHAb,運氣來得若有似無,62,0.485,0.103,1.0,...,1.0,0.0367,0.9590,0.000053,0.1070,0.210,140.595,270.761,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7857,2010,西洋,Prophetess Helen Nkume,Adam Olee Ebe Ino ?,3HF2QQOZC4MHRuMQA8MyrM,Adam Olee Ebe Ino ?,6,0.774,0.860,10.0,...,0.0,0.1730,0.1630,0.001000,0.0605,0.760,100.102,805.280,4.0,0
7858,2010,日語,Sunday 2PM,돌연변이,6WmL0e1FXxUjpwZKMThrVP,돌연변이,0,0.633,0.914,1.0,...,1.0,0.2560,0.0667,0.000000,0.2990,0.769,107.300,185.222,4.0,0
7859,2010,日語,Sunday 2PM,어떻게,6FR2t0ytRuak9p5bcZJTGQ,돌연변이,0,0.657,0.822,11.0,...,0.0,0.2320,0.0856,0.000000,0.0791,0.563,95.941,226.548,4.0,0
7860,2010,日語,Sunday 2PM,기도,3fIER6IBdl7GnQrZNFuoiE,돌연변이,0,0.385,0.856,11.0,...,0.0,0.2340,0.2410,0.000000,0.3650,0.519,86.628,291.070,5.0,0


In [23]:
# 輸出資料集
df_all.to_csv("D:\\全部歌曲.csv", index = False)